In [1]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense
import numpy as np
import os
import seaborn as sns
import pandas as pd
from math import e,sqrt
import matplotlib.pyplot as plt
from scipy.optimize import least_squares
from scipy.optimize import leastsq
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import r2_score

In [2]:
import numpy as np
def clean_dataset(df):
    assert isinstance(df, pd.DataFrame) 
    df.dropna(inplace=True)
    indices_to_keep = ~df.isin([np.nan, np.inf, -np.inf]).any(1)
    return df[indices_to_keep].astype(np.float64)

In [3]:
import pandas as pd

df = pd.read_csv('/content/smart_home_data.csv', sep=',')

df.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (0,27) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],Kitchen 14 [kW],Kitchen 38 [kW],Barn [kW],Well [kW],Microwave [kW],Living room [kW],Solar [kW],temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.0,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0,24.4,0.0


In [4]:
df=df[:-1]
df.tail()

,time,use [kW],gen [kW],House overall [kW],Dishwasher [kW],Furnace 1 [kW],Furnace 2 [kW],Home office [kW],Fridge [kW],Wine cellar [kW],Garage door [kW],Kitchen 12 [kW],Kitchen 14 [kW],Kitchen 38 [kW],Barn [kW],Well [kW],Microwave [kW],Living room [kW],Solar [kW],temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
503905,1452128305,1.601233,0.003183,1.601233,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,0.000150,0.000017,0.032283,0.000983,0.003800,0.000967,0.003183,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503906,1452128306,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,0.000100,0.000033,0.032200,0.000950,0.003800,0.000933,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503907,1452128307,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,0.000067,0.000000,0.032283,0.001000,0.003750,0.001000,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503908,1452128308,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,0.000517,0.000117,0.000017,0.032183,0.000950,0.003767,0.000950,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503909,1452128309,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,0.000517,0.000117,0.000017,0.032167,0.000950,0.003767,0.000950,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51


In [5]:
df.columns = [col.replace(' [kW]', '') for col in df.columns]
df.columns

Index(['time', 'use', 'gen', 'House overall', 'Dishwasher', 'Furnace 1',
       'Furnace 2', 'Home office', 'Fridge', 'Wine cellar', 'Garage door',
       'Kitchen 12', 'Kitchen 14', 'Kitchen 38', 'Barn', 'Well', 'Microwave',
       'Living room', 'Solar', 'temperature', 'icon', 'humidity', 'visibility',
       'summary', 'apparentTemperature', 'pressure', 'windSpeed', 'cloudCover',
       'windBearing', 'precipIntensity', 'dewPoint', 'precipProbability'],
      dtype='object')

In [6]:
df.iloc[np.r_[0:5,-5:0]]

,time,use,gen,House overall,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,Kitchen 12,Kitchen 14,Kitchen 38,Barn,Well,Microwave,Living room,Solar,temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
1,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
3,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
4,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
503905,1452128305,1.601233,0.003183,1.601233,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,0.000150,0.000017,0.032283,0.000983,0.003800,0.000967,0.003183,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503906,1452128306,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,0.000100,0.000033,0.032200,0.000950,0.003800,0.000933,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503907,1452128307,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,0.000067,0.000000,0.032283,0.001000,0.003750,0.001000,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503908,1452128308,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,0.000517,0.000117,0.000017,0.032183,0.000950,0.003767,0.000950,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
503909,1452128309,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,0.000517,0.000117,0.000017,0.032167,0.000950,0.003767,0.000950,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51


In [7]:
import time 
print(' start ' , time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(1451624400)))

 start  2016-01-01 05:00:00


In [8]:
time_index = pd.date_range('2016-01-01 05:00', periods=len(df),  freq='min')  
time_index = pd.DatetimeIndex(time_index)
df= df.set_index(time_index)
#df = df.drop(['time'], axis=1)
df.iloc[np.r_[0:5,-5:0]]


,time,use,gen,House overall,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,Kitchen 12,Kitchen 14,Kitchen 38,Barn,Well,Microwave,Living room,Solar,temperature,icon,humidity,visibility,summary,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
2016-01-01 05:00:00,1451624400,0.932833,0.003483,0.932833,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:01:00,1451624401,0.934333,0.003467,0.934333,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:02:00,1451624402,0.931817,0.003467,0.931817,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:03:00,1451624403,1.022050,0.003483,1.022050,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:04:00,1451624404,1.139400,0.003467,1.139400,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,clear-night,0.62,10.00,Clear,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-12-16 03:25:00,1452128305,1.601233,0.003183,1.601233,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,0.000150,0.000017,0.032283,0.000983,0.003800,0.000967,0.003183,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:26:00,1452128306,1.599333,0.003233,1.599333,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,0.000100,0.000033,0.032200,0.000950,0.003800,0.000933,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:27:00,1452128307,1.924267,0.003217,1.924267,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,0.000067,0.000000,0.032283,0.001000,0.003750,0.001000,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:28:00,1452128308,1.978200,0.003217,1.978200,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,0.000517,0.000117,0.000017,0.032183,0.000950,0.003767,0.000950,0.003217,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:29:00,1452128309,1.990950,0.003233,1.990950,0.000050,0.494700,0.634133,0.042100,0.004917,0.008133,0.012833,0.000517,0.000117,0.000017,0.032167,0.000950,0.003767,0.000950,0.003233,35.12,rain,0.86,8.74,Light Rain,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51


In [9]:
df=df.drop(['House overall','summary','icon'],axis=1)
df

,time,use,gen,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,Kitchen 12,Kitchen 14,Kitchen 38,Barn,Well,Microwave,Living room,Solar,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
2016-01-01 05:00:00,1451624400,0.932833,0.003483,0.000033,0.020700,0.061917,0.442633,0.124150,0.006983,0.013083,0.000417,0.000150,0.000000,0.031350,0.001017,0.004067,0.001517,0.003483,36.14,0.62,10.00,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:01:00,1451624401,0.934333,0.003467,0.000000,0.020717,0.063817,0.444067,0.124000,0.006983,0.013117,0.000417,0.000150,0.000000,0.031500,0.001017,0.004067,0.001650,0.003467,36.14,0.62,10.00,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:02:00,1451624402,0.931817,0.003467,0.000017,0.020700,0.062317,0.446067,0.123533,0.006983,0.013083,0.000433,0.000167,0.000017,0.031517,0.001000,0.004067,0.001650,0.003467,36.14,0.62,10.00,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:03:00,1451624403,1.022050,0.003483,0.000017,0.106900,0.068517,0.446583,0.123133,0.006983,0.013000,0.000433,0.000217,0.000000,0.031500,0.001017,0.004067,0.001617,0.003483,36.14,0.62,10.00,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
2016-01-01 05:04:00,1451624404,1.139400,0.003467,0.000133,0.236933,0.063983,0.446533,0.122850,0.006850,0.012783,0.000450,0.000333,0.000000,0.031500,0.001017,0.004067,0.001583,0.003467,36.14,0.62,10.00,29.26,1016.91,9.18,cloudCover,282.0,0.0000,24.40,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2016-12-16 03:25:00,1452128305,1.601233,0.003183,0.000050,0.085267,0.642417,0.041783,0.005267,0.008667,0.013483,0.000467,0.000150,0.000017,0.032283,0.000983,0.003800,0.000967,0.003183,35.12,0.86,8.74,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:26:00,1452128306,1.599333,0.003233,0.000050,0.104017,0.625033,0.041750,0.005233,0.008433,0.013433,0.000467,0.000100,0.000033,0.032200,0.000950,0.003800,0.000933,0.003233,35.12,0.86,8.74,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:27:00,1452128307,1.924267,0.003217,0.000033,0.422383,0.637733,0.042033,0.004983,0.008467,0.012933,0.000533,0.000067,0.000000,0.032283,0.001000,0.003750,0.001000,0.003217,35.12,0.86,8.74,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51
2016-12-16 03:28:00,1452128308,1.978200,0.003217,0.000050,0.495667,0.620367,0.042100,0.005333,0.008233,0.012817,0.000517,0.000117,0.000017,0.032183,0.000950,0.003767,0.000950,0.003217,35.12,0.86,8.74,29.45,1011.49,6.72,0.31,186.0,0.0101,31.27,0.51


In [10]:
df['cloudCover'].replace(['cloudCover'], method='bfill', inplace=True)
df['cloudCover'] = df['cloudCover'].astype('float')
df['cloudCover'].unique()

array([0.75, 0.  , 1.  , 0.31, 0.44, 0.13, 0.19, 0.25, 0.16, 0.21, 0.15,
       0.14, 0.27, 0.28, 0.17, 0.05, 0.1 , 0.26, 0.29, 0.11, 0.09, 0.12,
       0.06, 0.02, 0.08, 0.04, 0.35, 0.22, 0.23, 0.54, 0.39, 0.03, 0.07,
       0.76, 0.62, 0.18, 0.79, 0.48, 0.24, 0.57, 0.41, 0.78, 0.2 , 0.77,
       0.46, 0.55, 0.01, 0.51, 0.47, 0.5 , 0.4 , 0.3 , 0.43, 0.33, 0.6 ,
       0.68, 0.66, 0.45, 0.34, 0.52, 0.67, 0.49, 0.37, 0.36, 0.61, 0.38,
       0.42, 0.53, 0.63, 0.32, 0.56, 0.58, 0.72, 0.73, 0.71, 0.64, 0.59])

In [11]:
df = df.resample('10T').mean()
print("Shape of hourly data: {} --> n_rows = {}, n_cols = {}".format(df.shape, df.shape[0],df.shape[1]))

Shape of hourly data: (50391, 28) --> n_rows = 50391, n_cols = 28


In [12]:
df=df.reset_index(drop=True)

In [13]:
df.head()

,use,gen,Dishwasher,Furnace 1,Furnace 2,Home office,Fridge,Wine cellar,Garage door,Kitchen 12,Kitchen 14,Kitchen 38,Barn,Well,Microwave,Living room,Solar,temperature,humidity,visibility,apparentTemperature,pressure,windSpeed,cloudCover,windBearing,precipIntensity,dewPoint,precipProbability
0,1.251310,0.003450,0.000122,0.250567,0.167477,0.444067,0.122707,0.006920,0.012853,0.000447,0.000293,0.000003,0.031553,0.001018,0.004112,0.001587,0.003450,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0
1,0.875018,0.003440,0.000018,0.021088,0.239840,0.353613,0.042828,0.007120,0.013253,0.000612,0.000127,0.000008,0.031792,0.001007,0.004223,0.001620,0.003440,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0
2,1.254552,0.003225,0.000057,0.235440,0.353640,0.257390,0.004945,0.008863,0.012973,0.000670,0.000215,0.000012,0.045390,0.001082,0.108498,0.001533,0.003225,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0
3,1.025522,0.003060,0.000037,0.057535,0.173268,0.253203,0.004922,0.111477,0.013062,0.000647,0.000233,0.000010,0.045958,0.001083,0.004498,0.001230,0.003060,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0
4,0.961780,0.003252,0.000087,0.163970,0.241480,0.099610,0.008567,0.122230,0.012967,0.000665,0.000158,0.000008,0.046852,0.001058,0.004355,0.001430,0.003252,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0


In [14]:
def series_to_supervised_mimo(data, n_in, n_out, dropnan=True):
	"""
	Frame a time series as a supervised learning dataset.
	Arguments:
		data: Sequence of observations as a list or NumPy array.
		n_in: Number of lag observations as input (X).
		n_out: Number of observations as output (y).
		dropnan: Boolean whether or not to drop rows with NaN values.
	Returns:
		Pandas DataFrame of series framed for supervised learning.
	"""
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()
	# input sequence (t-n, ... t-1)
	for i in range(n_in, 0, -1):
		cols.append(df.shift(i))
		names += [(df.columns[j]+'(t-%d)' % (i)) for j in range(n_vars)]
	# forecast sequence (t, t+1, ... t+n)
	for i in range(0, n_out):
		cols.append(df.shift(-i))
		if i == 0:
			names += [(df.columns[j]+'(t)') for j in range(n_vars)]
		else:
			names += [(df.columns[j]+'%d(t+%d)' % (j, i)) for j in range(n_vars)]
	# put it all together
	agg = pd.concat(cols, axis=1)
	agg.columns = names
	# drop rows with NaN values
	if dropnan:
		agg.dropna(inplace=True)
	return agg

In [15]:
reframed = series_to_supervised_mimo(df, 1, 1)
reframed.head()

,use(t-1),gen(t-1),Dishwasher(t-1),Furnace 1(t-1),Furnace 2(t-1),Home office(t-1),Fridge(t-1),Wine cellar(t-1),Garage door(t-1),Kitchen 12(t-1),Kitchen 14(t-1),Kitchen 38(t-1),Barn(t-1),Well(t-1),Microwave(t-1),Living room(t-1),Solar(t-1),temperature(t-1),humidity(t-1),visibility(t-1),apparentTemperature(t-1),pressure(t-1),windSpeed(t-1),cloudCover(t-1),windBearing(t-1),precipIntensity(t-1),dewPoint(t-1),precipProbability(t-1),use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
1,1.251310,0.003450,0.000122,0.250567,0.167477,0.444067,0.122707,0.006920,0.012853,0.000447,0.000293,0.000003,0.031553,0.001018,0.004112,0.001587,0.003450,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.875018,0.003440,0.000018,0.021088,0.239840,0.353613,0.042828,0.007120,0.013253,0.000612,0.000127,0.000008,0.031792,0.001007,0.004223,0.001620,0.003440,36.140,0.620,10.0,29.260,1016.910,9.180,0.75,282.0,0.0,24.4,0.0
2,0.875018,0.003440,0.000018,0.021088,0.239840,0.353613,0.042828,0.007120,0.013253,0.000612,0.000127,0.000008,0.031792,0.001007,0.004223,0.001620,0.003440,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,1.254552,0.003225,0.000057,0.235440,0.353640,0.257390,0.004945,0.008863,0.012973,0.000670,0.000215,0.000012,0.045390,0.001082,0.108498,0.001533,0.003225,36.140,0.620,10.0,29.260,1016.910,9.180,0.75,282.0,0.0,24.4,0.0
3,1.254552,0.003225,0.000057,0.235440,0.353640,0.257390,0.004945,0.008863,0.012973,0.000670,0.000215,0.000012,0.045390,0.001082,0.108498,0.001533,0.003225,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,1.025522,0.003060,0.000037,0.057535,0.173268,0.253203,0.004922,0.111477,0.013062,0.000647,0.000233,0.000010,0.045958,0.001083,0.004498,0.001230,0.003060,36.140,0.620,10.0,29.260,1016.910,9.180,0.75,282.0,0.0,24.4,0.0
4,1.025522,0.003060,0.000037,0.057535,0.173268,0.253203,0.004922,0.111477,0.013062,0.000647,0.000233,0.000010,0.045958,0.001083,0.004498,0.001230,0.003060,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.961780,0.003252,0.000087,0.163970,0.241480,0.099610,0.008567,0.122230,0.012967,0.000665,0.000158,0.000008,0.046852,0.001058,0.004355,0.001430,0.003252,36.140,0.620,10.0,29.260,1016.910,9.180,0.75,282.0,0.0,24.4,0.0
5,0.961780,0.003252,0.000087,0.163970,0.241480,0.099610,0.008567,0.122230,0.012967,0.000665,0.000158,0.000008,0.046852,0.001058,0.004355,0.001430,0.003252,36.14,0.62,10.0,29.26,1016.91,9.18,0.75,282.0,0.0,24.4,0.0,0.896597,0.003415,0.000062,0.079185,0.375638,0.043002,0.043198,0.123493,0.013168,0.000592,0.000208,0.000015,0.031738,0.001005,0.004223,0.001628,0.003415,36.086,0.618,10.0,29.288,1016.778,9.002,0.75,282.6,0.0,24.3,0.0


In [16]:
x,y=reframed.loc[:,:],reframed.loc[:,:]

In [18]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_testx=x_test.loc[:,'use(t-1)':'precipProbability(t-1)']
df_trainy=y_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_original=y_test.loc[:,'use(t)':'precipProbability(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [19]:
def get_model(n_inputs, n_outputs):
  model = Sequential()
  model.add(Dense(128, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64,kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(32,kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
  model.compile(loss='mae', optimizer='adam')
  return model

In [20]:
model = get_model(n_inputs, n_outputs)

In [21]:
model.fit(df_trainx,df_trainy, verbose=0, epochs=100)

In [22]:
print(n_outputs,n_inputs)

28 28


In [23]:
y_p=model.predict(df_testx)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
0,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
1,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
2,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
3,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
4,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
12594,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
12595,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022
12596,0.610191,0.003972,0.000024,0.021523,0.066315,0.042551,0.032156,0.008054,0.013041,0.000551,0.000143,-0.000068,0.032324,0.000911,0.003741,0.001517,0.003939,50.262501,0.678498,9.998885,50.228546,118.046745,5.941967,0.115028,115.289162,0.000165,38.987385,0.000022


In [30]:
df_original

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
7169,0.972997,0.003312,0.000025,0.112272,0.253815,0.041660,0.124033,0.121798,0.012637,0.000403,0.000093,0.000002,0.067540,0.001040,0.004070,0.001573,0.003312,49.290,0.490,10.000,45.420,1012.170,9.260,0.210,286.0,0.00000,30.890,0.000
7832,0.443733,0.002712,0.000003,0.063600,0.103367,0.039658,0.005503,0.006772,0.012888,0.000680,0.000030,0.000007,0.030983,0.001017,0.004088,0.001417,0.002712,28.790,0.610,9.120,18.280,1002.730,13.040,0.310,291.0,0.00000,17.190,0.000
29597,0.035458,0.394792,0.000273,0.019887,0.081597,0.039203,0.005047,0.137357,0.012663,0.000768,0.000068,0.000010,0.028838,0.000998,0.004072,0.001638,0.394792,67.766,0.894,9.082,67.766,1018.952,4.680,0.310,44.8,0.01926,64.538,0.578
37737,0.331772,0.004402,0.000005,0.020672,0.065565,0.042662,0.005183,0.032122,0.013172,0.000782,0.000018,0.000008,0.032475,0.001000,0.003192,0.001668,0.004402,62.264,0.524,10.000,62.264,1031.170,9.958,0.092,66.4,0.00000,44.492,0.000
6374,1.171500,0.002903,0.000162,0.485255,0.320857,0.041507,0.083447,0.006665,0.013460,0.000448,0.000150,0.000008,0.031962,0.000952,0.003962,0.001410,0.002903,16.090,0.796,3.106,7.938,1029.378,5.396,0.862,34.4,0.00408,10.984,0.170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8048,0.842232,0.003093,0.000028,0.064040,0.350730,0.041325,0.124930,0.007285,0.013193,0.000348,0.000203,0.000018,0.032882,0.000968,0.003990,0.001520,0.003093,33.030,0.470,10.000,26.180,1013.210,7.880,0.030,191.0,0.00000,14.810,0.000
50328,0.590448,0.052390,0.000062,0.194900,0.232498,0.040217,0.005143,0.008127,0.012958,0.000705,0.000077,0.000010,0.029030,0.000998,0.003145,0.001282,0.052390,34.480,0.660,6.765,24.965,999.785,14.545,0.220,284.0,0.00175,24.455,0.045
13675,0.885382,0.003340,0.090672,0.176202,0.202145,0.041393,0.129542,0.006920,0.012780,0.000360,0.000020,0.000008,0.033055,0.001003,0.004068,0.001613,0.003340,40.840,0.780,9.920,36.450,996.860,6.480,0.070,254.0,0.00000,34.540,0.000
32455,0.282782,0.281353,0.000002,0.019975,0.068955,0.039120,0.117482,0.150485,0.012615,0.000575,0.000010,0.000015,0.028462,0.001002,0.004013,0.001498,0.281353,72.280,0.920,6.080,72.280,1008.900,4.360,0.500,163.0,0.06640,69.870,0.740


In [26]:
y_p.shape,df_original.shape

((12598, 28), (12598, 28))

In [27]:
import math
def cal_nrmse(rmse, y):
    x = max(y)-min(y)
    return (rmse/x)

In [28]:

col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)
 

  



In [29]:

import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('use(t)', 0.953, 0.48, -0.074, 1.354)
Results: ('gen(t)', 0.144, 0.072, -0.32, 0.605)
Results: ('Dishwasher(t)', 0.171, 0.029, -0.03, 1549498621.11)
Results: ('Furnace 1(t)', 0.162, 0.08, -0.307, 0.418)
Results: ('Furnace 2(t)', 0.158, 0.074, -0.26, 0.637)
Results: ('Home office(t)', 0.114, 0.043, -0.145, 0.438)
Results: ('Fridge(t)', 0.077, 0.057, -0.203, 2.861)
Results: ('Wine cellar(t)', 0.066, 0.035, -0.371, 0.604)
Results: ('Garage door(t)', 0.01, 0.001, -0.011, 0.15)
Results: ('Kitchen 12(t)', 0.019, 0.002, -0.016, 984774378.932)
Results: ('Kitchen 14(t)', 0.047, 0.007, -0.023, 408428803.031)
Results: ('Kitchen 38(t)', 0.0, 0.0, -491.18, 608462532.799)
Results: ('Barn(t)', 0.128, 0.028, -0.04, 0.298)
Results: ('Well(t)', 0.07, 0.015, -0.049, 1628951017.792)
Results: ('Microwave(t)', 0.061, 0.007, -0.012, 6686635062.909)
Results: ('Living room(t)', 0.101, 0.035, -0.135, 2711035900.35)
Results: ('Solar(t)', 0.144, 0.072, -0.32, 0.601)
Results: ('temperature(t)', 19.286, 1

,variable,rmse,mae,r2,mape
0,use(t),0.953,0.480,-0.074,1.354000e+00
1,gen(t),0.144,0.072,-0.320,6.050000e-01
2,Dishwasher(t),0.171,0.029,-0.030,1.549499e+09
3,Furnace 1(t),0.162,0.080,-0.307,4.180000e-01
4,Furnace 2(t),0.158,0.074,-0.260,6.370000e-01
5,Home office(t),0.114,0.043,-0.145,4.380000e-01
6,Fridge(t),0.077,0.057,-0.203,2.861000e+00
7,Wine cellar(t),0.066,0.035,-0.371,6.040000e-01
8,Garage door(t),0.010,0.001,-0.011,1.500000e-01
9,Kitchen 12(t),0.019,0.002,-0.016,9.847744e+08


##**MinMaxScaler**

In [31]:
x,y=reframed.loc[:,:],reframed.loc[:,:]

In [32]:
from sklearn.preprocessing import MinMaxScaler
scalerx= MinMaxScaler()
scalery=MinMaxScaler()


In [41]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'use(t-1)':'precipProbability(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'use(t)':'precipProbability(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [42]:
def get_model(n_inputs, n_outputs):
  model = Sequential()
  model.add(Dense(128, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(64,kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(32,kernel_initializer='he_uniform', activation='relu'))
  model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
  model.compile(loss='mae', optimizer='adam')
  return model

In [43]:
model = get_model(n_inputs, n_outputs)

In [44]:
model.fit(df_trainx_scaled,df_trainy_scaled, verbose=0, epochs=100)

In [45]:
y_p=model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
0,0.093595,0.001992,-0.000154,0.065735,0.426451,0.055366,0.196422,0.014556,0.018832,0.001191,0.001594,0.405727,0.012848,0.000518,0.003211,0.004233,0.001963,0.585093,0.443689,1.003834,0.576590,0.466295,0.425647,0.198538,0.761059,-0.000827,0.574131,-0.004124
1,0.042664,0.002192,-0.000099,0.034279,0.132046,0.055720,0.008026,0.002519,0.018806,0.001577,0.001856,0.404029,0.013977,0.000695,0.003328,0.004002,0.001997,0.393804,0.566571,0.913576,0.394579,0.293899,0.566329,0.307216,0.804194,-0.000701,0.430969,-0.003399
2,0.003905,0.725448,0.000364,0.004395,0.106545,0.050091,0.005349,0.031281,0.020002,0.000774,0.003415,0.158010,0.012036,0.000432,0.003684,0.003224,0.724123,0.759953,0.902530,0.890367,0.760713,0.587356,0.189905,0.300744,0.125392,0.106929,0.904340,0.745962
3,0.029105,0.005225,0.000344,0.010269,0.085986,0.054578,0.007528,0.019658,0.019715,0.001436,0.002251,0.305436,0.013801,0.000862,0.003580,0.003419,0.004175,0.699944,0.483626,1.004121,0.684955,0.803527,0.484874,0.054134,0.160931,-0.000616,0.700226,-0.003828
4,0.160643,0.004279,0.000543,0.137100,0.862500,0.055615,0.197245,0.012793,0.020631,0.000210,0.003595,0.458889,0.015737,0.001032,0.004070,0.003094,0.002520,0.273033,0.763727,0.476236,0.312130,0.780981,0.200597,0.304557,0.081872,0.000389,0.366211,0.005863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593,0.082481,0.001991,0.000113,0.091207,0.583475,0.053036,0.164267,0.004450,0.019729,0.000716,0.002412,0.555288,0.014388,0.000783,0.003797,0.003362,0.001174,0.435522,0.401104,1.004572,0.446755,0.479286,0.338109,0.026576,0.522690,-0.000850,0.415002,-0.004208
12594,0.077690,0.063795,-0.000170,0.098831,0.589505,0.053173,0.006874,0.007109,0.019298,0.001350,0.001805,0.456405,0.013621,0.000757,0.003355,0.003105,0.063490,0.457869,0.591338,0.590922,0.440072,0.233582,0.706891,-0.004288,0.775886,0.000683,0.495147,0.005651
12595,0.094103,0.002996,-0.000050,0.078173,0.470923,0.055795,0.006856,0.009396,0.019833,0.001258,0.002401,0.304576,0.014731,0.000881,0.003857,0.003283,0.002095,0.507064,0.764167,0.995405,0.516444,0.188203,0.276599,0.066323,0.700902,-0.000828,0.613590,-0.004120
12596,0.028178,0.516114,0.000137,0.003281,0.089728,0.051086,0.232113,0.029693,0.019038,0.000578,0.003027,0.307283,0.011561,0.000220,0.003234,0.003998,0.515498,0.802592,0.940663,0.603371,0.797505,0.408960,0.184303,0.492326,0.445383,0.126055,0.947331,0.879238


In [46]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
0,1.051322,0.001286,-0.000214,0.116519,0.331582,0.040478,0.130636,0.017267,0.013893,0.000548,0.001424,0.000014,0.032930,0.000513,0.005531,0.001684,0.001270,49.590481,0.507135,10.037301,44.721786,1012.540466,9.751583,0.198538,273.220184,-0.000158,31.740517,-0.003464
1,0.479663,0.001396,-0.000137,0.060872,0.102717,0.040737,0.005434,0.003002,0.013875,0.000726,0.001658,0.000013,0.035815,0.000689,0.005733,0.001592,0.001289,29.244997,0.611586,9.159100,20.477938,1002.876038,12.974596,0.307216,288.705536,-0.000134,17.033484,-0.002855
2,0.044636,0.399341,0.000506,0.008008,0.082893,0.036629,0.003655,0.037087,0.014754,0.000356,0.003050,0.000005,0.030853,0.000428,0.006347,0.001282,0.398612,68.188591,0.897150,8.933269,69.246956,1019.327148,4.350731,0.300744,45.015839,0.020423,65.662842,0.626608
3,0.327479,0.003065,0.000478,0.018400,0.066910,0.039903,0.005103,0.023313,0.014543,0.000661,0.002011,0.000010,0.035365,0.000855,0.006169,0.001360,0.002487,61.806011,0.541082,10.040094,59.155972,1031.445679,11.108455,0.054134,57.774086,-0.000118,44.694214,-0.003216
4,1.803884,0.002544,0.000754,0.242764,0.670557,0.040660,0.131183,0.015177,0.015216,0.000097,0.003211,0.000015,0.040316,0.001023,0.007012,0.001230,0.001576,16.399841,0.779168,4.903778,9.495741,1030.181763,4.595683,0.304557,29.391968,0.000074,10.380816,0.004925
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593,0.926577,0.001286,0.000157,0.161578,0.453649,0.038778,0.109267,0.005291,0.014553,0.000330,0.002155,0.000019,0.036866,0.000776,0.006541,0.001337,0.000836,33.682083,0.470939,10.044484,27.427746,1013.268799,7.746078,0.026576,187.645737,-0.000162,15.393125,-0.003535
12594,0.872809,0.035291,-0.000236,0.175065,0.458336,0.038878,0.004668,0.008442,0.014236,0.000621,0.001612,0.000015,0.034907,0.000751,0.005780,0.001235,0.035123,36.058918,0.632638,6.019670,26.537647,999.494629,16.194881,-0.004288,278.543091,0.000130,23.626432,0.004747
12595,1.057031,0.001838,-0.000069,0.138522,0.366153,0.040791,0.004657,0.011152,0.014629,0.000579,0.002144,0.000010,0.037744,0.000873,0.006645,0.001306,0.001343,41.291351,0.779542,9.955296,36.710335,996.950623,6.336890,0.066323,251.623856,-0.000158,35.794140,-0.003461
12596,0.317079,0.284163,0.000190,0.006037,0.069819,0.037356,0.154355,0.035206,0.014045,0.000266,0.002704,0.000010,0.029639,0.000218,0.005572,0.001590,0.283824,72.723679,0.929563,6.140804,74.147690,1009.326355,4.222376,0.492326,159.892609,0.024077,70.079300,0.738560


In [40]:
df_original

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
7169,0.972997,0.003312,0.000025,0.112272,0.253815,0.041660,0.124033,0.121798,0.012637,0.000403,0.000093,0.000002,0.067540,0.001040,0.004070,0.001573,0.003312,49.290,0.490,10.000,45.420,1012.170,9.260,0.210,286.0,0.00000,30.890,0.000
7832,0.443733,0.002712,0.000003,0.063600,0.103367,0.039658,0.005503,0.006772,0.012888,0.000680,0.000030,0.000007,0.030983,0.001017,0.004088,0.001417,0.002712,28.790,0.610,9.120,18.280,1002.730,13.040,0.310,291.0,0.00000,17.190,0.000
29597,0.035458,0.394792,0.000273,0.019887,0.081597,0.039203,0.005047,0.137357,0.012663,0.000768,0.000068,0.000010,0.028838,0.000998,0.004072,0.001638,0.394792,67.766,0.894,9.082,67.766,1018.952,4.680,0.310,44.8,0.01926,64.538,0.578
37737,0.331772,0.004402,0.000005,0.020672,0.065565,0.042662,0.005183,0.032122,0.013172,0.000782,0.000018,0.000008,0.032475,0.001000,0.003192,0.001668,0.004402,62.264,0.524,10.000,62.264,1031.170,9.958,0.092,66.4,0.00000,44.492,0.000
6374,1.171500,0.002903,0.000162,0.485255,0.320857,0.041507,0.083447,0.006665,0.013460,0.000448,0.000150,0.000008,0.031962,0.000952,0.003962,0.001410,0.002903,16.090,0.796,3.106,7.938,1029.378,5.396,0.862,34.4,0.00408,10.984,0.170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8048,0.842232,0.003093,0.000028,0.064040,0.350730,0.041325,0.124930,0.007285,0.013193,0.000348,0.000203,0.000018,0.032882,0.000968,0.003990,0.001520,0.003093,33.030,0.470,10.000,26.180,1013.210,7.880,0.030,191.0,0.00000,14.810,0.000
50328,0.590448,0.052390,0.000062,0.194900,0.232498,0.040217,0.005143,0.008127,0.012958,0.000705,0.000077,0.000010,0.029030,0.000998,0.003145,0.001282,0.052390,34.480,0.660,6.765,24.965,999.785,14.545,0.220,284.0,0.00175,24.455,0.045
13675,0.885382,0.003340,0.090672,0.176202,0.202145,0.041393,0.129542,0.006920,0.012780,0.000360,0.000020,0.000008,0.033055,0.001003,0.004068,0.001613,0.003340,40.840,0.780,9.920,36.450,996.860,6.480,0.070,254.0,0.00000,34.540,0.000
32455,0.282782,0.281353,0.000002,0.019975,0.068955,0.039120,0.117482,0.150485,0.012615,0.000575,0.000010,0.000015,0.028462,0.001002,0.004013,0.001498,0.281353,72.280,0.920,6.080,72.280,1008.900,4.360,0.500,163.0,0.06640,69.870,0.740


In [47]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)
 

In [48]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('use(t)', 0.676, 0.301, 0.459, 0.472)
Results: ('gen(t)', 0.028, 0.012, 0.951, 0.437)
Results: ('Dishwasher(t)', 0.171, 0.03, -0.029, 17921862417.073)
Results: ('Furnace 1(t)', 0.137, 0.073, 0.075, 0.88)
Results: ('Furnace 2(t)', 0.125, 0.059, 0.219, 0.577)
Results: ('Home office(t)', 0.114, 0.044, -0.136, 0.432)
Results: ('Fridge(t)', 0.059, 0.03, 0.304, 1.498)
Results: ('Wine cellar(t)', 0.056, 0.033, 0.01, 1.487)
Results: ('Garage door(t)', 0.01, 0.003, 0.009, 0.254)
Results: ('Kitchen 12(t)', 0.019, 0.002, -0.016, 1293162137.052)
Results: ('Kitchen 14(t)', 0.047, 0.009, -0.011, 5599246073.904)
Results: ('Kitchen 38(t)', 0.0, 0.0, -0.622, 62017078.541)
Results: ('Barn(t)', 0.126, 0.029, -0.019, 0.379)
Results: ('Well(t)', 0.071, 0.015, -0.05, 1165123853.537)
Results: ('Microwave(t)', 0.06, 0.009, -0.004, 10377036036.772)
Results: ('Living room(t)', 0.101, 0.035, -0.134, 2548539997.037)
Results: ('Solar(t)', 0.028, 0.012, 0.951, 0.511)
Results: ('temperature(t)', 0.989, 0.7

,variable,rmse,mae,r2,mape
0,use(t),0.676,0.301,0.459,4.720000e-01
1,gen(t),0.028,0.012,0.951,4.370000e-01
2,Dishwasher(t),0.171,0.030,-0.029,1.792186e+10
3,Furnace 1(t),0.137,0.073,0.075,8.800000e-01
4,Furnace 2(t),0.125,0.059,0.219,5.770000e-01
5,Home office(t),0.114,0.044,-0.136,4.320000e-01
6,Fridge(t),0.059,0.030,0.304,1.498000e+00
7,Wine cellar(t),0.056,0.033,0.010,1.487000e+00
8,Garage door(t),0.010,0.003,0.009,2.540000e-01
9,Kitchen 12(t),0.019,0.002,-0.016,1.293162e+09


##MinMaxScaler+RandomizedSearchCV

In [49]:
x,y=reframed.loc[:,:],reframed.loc[:,:]

In [50]:
from sklearn.preprocessing import MinMaxScaler
scalerx= MinMaxScaler()
scalery=MinMaxScaler()


In [51]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42)
# Get the number of inputs and outputs from the dataset
df_trainx=x_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_trainx_scaled = scalerx.fit_transform(df_trainx)
df_testx=x_test.loc[:,'use(t-1)':'precipProbability(t-1)']
df_testx_scaled = scalerx.transform(df_testx)
df_trainy=y_train.loc[:,'use(t-1)':'precipProbability(t-1)']
df_trainy_scaled=scalery.fit_transform(df_trainy)

df_original=y_test.loc[:,'use(t)':'precipProbability(t)']

n_inputs, n_outputs = df_trainx.shape[1], df_trainy.shape[1]

In [52]:
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
def get_model(n_inputs=n_inputs, n_outputs=n_outputs,neurons1=32,neurons2=32,neurons3=32,batch_size=32,activation='relu'):
  model = Sequential()
  model.add(Dense(neurons1, input_dim=n_inputs, kernel_initializer='he_uniform', activation=activation))
  model.add(Dense(neurons2,kernel_initializer='he_uniform', activation=activation))
  model.add(Dense(neurons3,kernel_initializer='he_uniform', activation=activation))
  model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
  model.compile(loss='mse', optimizer='adam')
  early_stopping = EarlyStopping(monitor="loss", patience = 5)# early stop patience
  history = model.fit(df_trainx_scaled,df_trainy_scaled,
              batch_size=batch_size,
              epochs=20,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    
  return model

In [53]:
clf = KerasRegressor(build_fn=get_model, verbose=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [54]:
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'activation': ['relu','tanh'],
    'batch_size': [16,32,64],
    'neurons1':sp_randint(10,100),
    'neurons2':sp_randint(10,100),
    'neurons3':sp_randint(10,100)
    #'epochs':[20,50,100,200],
   
}

Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=10,cv=3,scoring='neg_mean_squared_error')

hyperparameters_tuning = Random.fit(df_trainx_scaled,df_trainy_scaled)
print('Best Parameters = {}'.format(hyperparameters_tuning.best_params_))

tuned_model = hyperparameters_tuning.best_estimator_

Best Parameters = {'activation': 'relu', 'batch_size': 32, 'neurons1': 63, 'neurons2': 53, 'neurons3': 56}


In [55]:
y_p=tuned_model.predict(df_testx_scaled)
columns=df_original.columns
y_p=pd.DataFrame(y_p,columns=columns)
y_p

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
0,0.095969,0.004639,0.001774,0.057967,0.428561,0.057197,0.194822,0.089144,0.022129,0.050002,0.000214,0.400669,0.026504,-0.001948,0.008158,0.003321,0.004639,0.578189,0.442555,1.001395,0.579755,0.466881,0.431886,0.200441,0.770380,0.009145,0.562142,-0.001863
1,0.041024,0.003886,0.001493,0.035779,0.132226,0.057154,0.009693,0.006681,0.022324,0.017229,0.000105,0.400609,0.013474,0.000413,0.006314,0.000290,0.003886,0.384419,0.566665,0.912524,0.396177,0.291299,0.571646,0.309253,0.812424,-0.003123,0.419221,0.002720
2,0.004541,0.726575,0.000771,0.008201,0.102269,0.054628,0.006841,0.077440,0.022194,0.048840,0.001284,0.148591,0.009682,-0.001361,0.000083,0.001420,0.726575,0.751288,0.893397,0.885802,0.760131,0.580688,0.194599,0.312935,0.133296,0.187814,0.894778,0.737884
3,0.027857,0.007156,0.000871,0.011251,0.086186,0.058517,0.006179,0.021636,0.021848,0.022794,0.001244,0.300224,0.014272,-0.002037,0.005359,0.002824,0.007156,0.700160,0.481735,1.001108,0.694677,0.800739,0.488551,0.058170,0.166863,0.000155,0.691949,0.000060
4,0.163514,0.002725,-0.000337,0.270698,0.869465,0.060777,0.199678,-0.006728,0.021020,0.045590,0.000232,0.451336,0.023094,-0.000607,0.006298,0.001206,0.002725,0.264065,0.763385,0.471936,0.313110,0.776463,0.211665,0.310976,0.089336,0.019413,0.356968,0.007478
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593,0.083708,0.004164,0.002640,0.037512,0.587320,0.058359,0.162622,0.000317,0.022278,0.026049,0.000287,0.550602,0.013992,-0.001309,0.004959,0.002693,0.004164,0.427308,0.401699,1.001382,0.449312,0.480221,0.344550,0.027924,0.532566,0.004811,0.398195,-0.000494
12594,0.078564,0.065603,0.001010,0.125796,0.592494,0.058133,0.009011,0.004843,0.021175,0.024145,0.001261,0.450983,0.011986,0.000738,0.004207,-0.001628,0.065603,0.449123,0.590868,0.589600,0.443692,0.231288,0.718420,0.000047,0.787445,0.014106,0.483996,0.011515
12595,0.094309,0.004297,0.084748,0.147930,0.472905,0.076834,0.007802,0.000616,0.021117,0.031916,0.000795,0.300735,0.013294,-0.000724,0.007806,0.000363,0.004297,0.496394,0.760832,0.994367,0.517136,0.187766,0.283845,0.068628,0.708650,0.001343,0.603927,0.002370
12596,0.027627,0.518757,0.000242,0.010993,0.088210,0.053400,0.235239,0.119899,0.021585,0.058133,-0.001818,0.298041,0.010891,-0.002711,0.031922,0.001435,0.518756,0.795400,0.929753,0.596058,0.797149,0.400926,0.194499,0.502539,0.454455,0.246870,0.939611,0.907027


In [56]:
y_p_inv = scalery.inverse_transform(y_p)
columns=df_original.columns
y_p_inv=pd.DataFrame(y_p_inv,columns=columns)
y_p_inv

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
0,1.077977,0.002742,0.002463,0.102777,0.333221,0.041814,0.129573,0.105659,0.016317,0.023021,0.000191,0.000013,0.067842,-0.001930,0.014055,0.001321,0.002742,48.856220,0.506172,10.013575,45.143303,1012.573242,9.894516,0.200441,276.566498,0.001747,30.508831,-0.001565
1,0.461257,0.002328,0.002073,0.063527,0.102856,0.041782,0.006542,0.007934,0.016461,0.007932,0.000094,0.000013,0.034529,0.000409,0.010878,0.000115,0.002328,28.246790,0.611666,9.148858,20.690752,1002.730225,13.096407,0.309253,291.660187,-0.000596,15.826523,0.002285
2,0.051764,0.399961,0.001070,0.014741,0.079569,0.039939,0.004646,0.091789,0.016364,0.022486,0.001146,0.000005,0.024835,-0.001349,0.000144,0.000565,0.399961,67.266960,0.889388,8.888849,69.169449,1018.953308,4.458253,0.312935,47.853199,0.035872,64.680511,0.619823
3,0.313472,0.004127,0.001209,0.020137,0.067066,0.042777,0.004207,0.025657,0.016111,0.010494,0.001111,0.000010,0.036569,-0.002018,0.009233,0.001123,0.004127,61.829037,0.539475,10.010783,60.450966,1031.289429,11.192698,0.058170,59.903683,0.000030,43.843960,0.000051
4,1.836106,0.001689,-0.000468,0.479098,0.675972,0.044426,0.132800,-0.007957,0.015502,0.020990,0.000207,0.000015,0.059123,-0.000602,0.010851,0.000480,0.001689,15.446001,0.778877,4.861942,9.626260,1029.928589,4.849242,0.310976,32.071796,0.003708,9.431364,0.006281
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12593,0.940353,0.002481,0.003665,0.066592,0.456638,0.042662,0.108174,0.000392,0.016426,0.011993,0.000256,0.000018,0.035855,-0.001297,0.008545,0.001071,0.002481,32.808521,0.471445,10.013446,27.768332,1013.321228,7.893643,0.027924,191.191238,0.000919,13.666618,-0.000415
12594,0.882622,0.036286,0.001402,0.222767,0.460660,0.042497,0.006088,0.005757,0.015616,0.011116,0.001126,0.000015,0.030727,0.000731,0.007248,-0.000647,0.036286,35.128693,0.632238,6.006806,27.019754,999.365967,16.459003,0.000047,282.692749,0.002694,22.480925,0.009672
12595,1.059338,0.002554,0.117674,0.261921,0.367693,0.056141,0.005285,0.000747,0.015573,0.014694,0.000710,0.000010,0.034070,-0.000717,0.013449,0.000144,0.002554,40.156513,0.776708,9.945189,36.802486,996.926147,6.502887,0.068628,254.405243,0.000256,34.801388,0.001991
12596,0.310895,0.285617,0.000336,0.019680,0.068640,0.039044,0.156433,0.142107,0.015917,0.026764,-0.001623,0.000010,0.027927,-0.002686,0.054999,0.000571,0.285617,71.958748,0.920290,6.069644,74.100235,1008.875854,4.455973,0.502539,163.149292,0.047152,69.286232,0.761903


In [57]:
df_original

,use(t),gen(t),Dishwasher(t),Furnace 1(t),Furnace 2(t),Home office(t),Fridge(t),Wine cellar(t),Garage door(t),Kitchen 12(t),Kitchen 14(t),Kitchen 38(t),Barn(t),Well(t),Microwave(t),Living room(t),Solar(t),temperature(t),humidity(t),visibility(t),apparentTemperature(t),pressure(t),windSpeed(t),cloudCover(t),windBearing(t),precipIntensity(t),dewPoint(t),precipProbability(t)
7169,0.972997,0.003312,0.000025,0.112272,0.253815,0.041660,0.124033,0.121798,0.012637,0.000403,0.000093,0.000002,0.067540,0.001040,0.004070,0.001573,0.003312,49.290,0.490,10.000,45.420,1012.170,9.260,0.210,286.0,0.00000,30.890,0.000
7832,0.443733,0.002712,0.000003,0.063600,0.103367,0.039658,0.005503,0.006772,0.012888,0.000680,0.000030,0.000007,0.030983,0.001017,0.004088,0.001417,0.002712,28.790,0.610,9.120,18.280,1002.730,13.040,0.310,291.0,0.00000,17.190,0.000
29597,0.035458,0.394792,0.000273,0.019887,0.081597,0.039203,0.005047,0.137357,0.012663,0.000768,0.000068,0.000010,0.028838,0.000998,0.004072,0.001638,0.394792,67.766,0.894,9.082,67.766,1018.952,4.680,0.310,44.8,0.01926,64.538,0.578
37737,0.331772,0.004402,0.000005,0.020672,0.065565,0.042662,0.005183,0.032122,0.013172,0.000782,0.000018,0.000008,0.032475,0.001000,0.003192,0.001668,0.004402,62.264,0.524,10.000,62.264,1031.170,9.958,0.092,66.4,0.00000,44.492,0.000
6374,1.171500,0.002903,0.000162,0.485255,0.320857,0.041507,0.083447,0.006665,0.013460,0.000448,0.000150,0.000008,0.031962,0.000952,0.003962,0.001410,0.002903,16.090,0.796,3.106,7.938,1029.378,5.396,0.862,34.4,0.00408,10.984,0.170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8048,0.842232,0.003093,0.000028,0.064040,0.350730,0.041325,0.124930,0.007285,0.013193,0.000348,0.000203,0.000018,0.032882,0.000968,0.003990,0.001520,0.003093,33.030,0.470,10.000,26.180,1013.210,7.880,0.030,191.0,0.00000,14.810,0.000
50328,0.590448,0.052390,0.000062,0.194900,0.232498,0.040217,0.005143,0.008127,0.012958,0.000705,0.000077,0.000010,0.029030,0.000998,0.003145,0.001282,0.052390,34.480,0.660,6.765,24.965,999.785,14.545,0.220,284.0,0.00175,24.455,0.045
13675,0.885382,0.003340,0.090672,0.176202,0.202145,0.041393,0.129542,0.006920,0.012780,0.000360,0.000020,0.000008,0.033055,0.001003,0.004068,0.001613,0.003340,40.840,0.780,9.920,36.450,996.860,6.480,0.070,254.0,0.00000,34.540,0.000
32455,0.282782,0.281353,0.000002,0.019975,0.068955,0.039120,0.117482,0.150485,0.012615,0.000575,0.000010,0.000015,0.028462,0.001002,0.004013,0.001498,0.281353,72.280,0.920,6.080,72.280,1008.900,4.360,0.500,163.0,0.06640,69.870,0.740


In [58]:
col=df_original.columns
result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]
    }
    
final_result = {
        
         "rmse": [],
         
         "mae": [],
         "r2": [],
         "mape":[],
         "variable":[]}
for col in col: 
  original = df_original[col].values
  forecast = y_p_inv[col].values
  mae = round(mean_absolute_error(original,forecast),3)
  r2 = round(r2_score(original,forecast),3)
  rmse = round(mean_squared_error(original,forecast,squared=False),3)             
  mape = round(mean_absolute_percentage_error(original,forecast),3)

  result["rmse"].append(rmse)
  result["mae"].append(mae)
  result["r2"].append(r2)
  result["mape"].append(mape)
  
  result["variable"].append(col)
        
measures = pd.DataFrame(result)

In [59]:
import statistics
final_result = {
    "variable": [],
    "rmse": [],
    # "nrmse": [],
    "mae": [],
    "r2": [],
    "mape":[]
}


var = measures.groupby("variable")

for col in columns:
    
    var_agr = var.get_group(col)
           
    rmse = round(statistics.mean(var_agr.loc[:,'rmse']),3)
    # nrmse = round(statistics.mean(var_agr.loc[:,'nrmse']),3)
    mae = round(statistics.mean(var_agr.loc[:,'mae']),3)
    r2 = round(statistics.mean(var_agr.loc[:,'r2']),3)
    mape = round(statistics.mean(var_agr.loc[:,'mape']),3)

    final_result["variable"].append(col)
    final_result["rmse"].append(rmse)
    # final_result["nrmse"].append(nrmse)
    final_result["mae"].append(mae)
    final_result["r2"].append(r2)
    final_result["mape"].append(mape)
        
    print(f'Results: {(col,rmse,mae,r2,mape)}')
        
        
final_measures_kpca = pd.DataFrame(final_result) 

print("Statistics MIMO (test): ")
final_measures_kpca

Results: ('use(t)', 0.677, 0.305, 0.458, 0.437)
Results: ('gen(t)', 0.028, 0.011, 0.951, 0.282)
Results: ('Dishwasher(t)', 0.145, 0.028, 0.257, 99710927205.892)
Results: ('Furnace 1(t)', 0.107, 0.05, 0.428, 0.684)
Results: ('Furnace 2(t)', 0.125, 0.059, 0.214, 0.364)
Results: ('Home office(t)', 0.039, 0.01, 0.869, 0.116)
Results: ('Fridge(t)', 0.059, 0.03, 0.302, 1.531)
Results: ('Wine cellar(t)', 0.03, 0.016, 0.707, 1.183)
Results: ('Garage door(t)', 0.01, 0.004, -0.046, 0.411)
Results: ('Kitchen 12(t)', 0.022, 0.016, -0.382, 17491689309.597)
Results: ('Kitchen 14(t)', 0.061, 0.013, -0.696, 1763498757.034)
Results: ('Kitchen 38(t)', 0.0, 0.0, -0.618, 59805635.441)
Results: ('Barn(t)', 0.095, 0.021, 0.423, 0.3)
Results: ('Well(t)', 0.088, 0.027, -0.652, 2071642060.4)
Results: ('Microwave(t)', 0.075, 0.022, -0.566, 11909514798.975)
Results: ('Living room(t)', 0.044, 0.01, 0.786, 2796882339.839)
Results: ('Solar(t)', 0.028, 0.011, 0.951, 0.282)
Results: ('temperature(t)', 1.027, 0.708, 0

,variable,rmse,mae,r2,mape
0,use(t),0.677,0.305,0.458,4.370000e-01
1,gen(t),0.028,0.011,0.951,2.820000e-01
2,Dishwasher(t),0.145,0.028,0.257,9.971093e+10
3,Furnace 1(t),0.107,0.050,0.428,6.840000e-01
4,Furnace 2(t),0.125,0.059,0.214,3.640000e-01
5,Home office(t),0.039,0.010,0.869,1.160000e-01
6,Fridge(t),0.059,0.030,0.302,1.531000e+00
7,Wine cellar(t),0.030,0.016,0.707,1.183000e+00
8,Garage door(t),0.010,0.004,-0.046,4.110000e-01
9,Kitchen 12(t),0.022,0.016,-0.382,1.749169e+10
